In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import random

# ----- 1. Set Random Seeds for Reproducibility -----
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# ----- 2. Load Data -----
df = pd.read_csv('dataset.csv')
# Drop rows where critical columns are missing
df = df.dropna(subset=['d18O', 'd13C', 'MARBLE GROUP'])

# ----- 3. Encode Labels -----
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['MARBLE GROUP'])

# Prepare features
features = ['d18O', 'd13C']
X = df[features].values
y = df['Label'].values

# ----- 4. Train / Test Split -----
# We create an 80/20 train/test split.
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# Further split the training set into train/validation, e.g. 90/10
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.1, random_state=SEED, stratify=y_train_full
)

# ----- 5. Scale Features -----
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# ----- 6. Create PyTorch Datasets -----
class MarbleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = MarbleDataset(X_train_scaled, y_train)
val_dataset   = MarbleDataset(X_val_scaled,   y_val)
test_dataset  = MarbleDataset(X_test_scaled,  y_test)

# ----- 7. Dataloaders -----
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# ----- 8. Define Model with BatchNorm & Dropout -----
class MarbleNet(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_size=16, dropout=0.2):
        super(MarbleNet, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        return x

input_dim = X_train_scaled.shape[1]
num_classes = len(np.unique(y))
model = MarbleNet(input_dim, num_classes, hidden_size=16, dropout=0.2)

# ----- 9. Training Setup -----
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # reduce LR every 30 epochs by 10x

# ----- 10. Training Loop with Early Stopping -----
def evaluate(dataloader):
    """Compute the average loss and accuracy for a given dataloader."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)

            total_loss += loss.item() * batch_y.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_y).sum().item()
            total_samples += batch_y.size(0)

    avg_loss = total_loss / total_samples
    accuracy = correct / total_samples
    return avg_loss, accuracy

num_epochs = 100
best_val_acc = 0.0
patience = 100 # stop if no improvement for these many epochs
counter_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_train_samples = 0
    
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_y.size(0)
        total_train_samples += batch_y.size(0)

    # Step the scheduler
    scheduler.step()

    # Compute training metrics
    train_loss = running_loss / total_train_samples
    
    # Compute validation metrics
    val_loss, val_acc = evaluate(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Acc: {val_acc*100:.2f}%")

    # Early Stopping Check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter_no_improve = 0
        best_model_state = model.state_dict()  # save best model
    else:
        counter_no_improve += 1
        if counter_no_improve >= patience:
            print("Early stopping triggered.")
            break

# ----- 11. Load the Best Model -----
model.load_state_dict(best_model_state)

# ----- 12. Final Evaluation on Test Set (Top-1 & Top-3) -----
model.eval()
correct_top1, correct_top3 = 0, 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)

        # --- Top-1 Accuracy ---
        _, predicted_top1 = torch.max(outputs, 1)
        correct_top1 += (predicted_top1 == batch_y).sum().item()
        
        # --- Top-3 Accuracy ---
        # Get the indices of the top 3 probabilities for each sample
        _, top3_indices = torch.topk(outputs, 3, dim=1)
        for i in range(batch_y.size(0)):
            if batch_y[i] in top3_indices[i]:
                correct_top3 += 1
        
        total += batch_y.size(0)

top1_accuracy = 100.0 * correct_top1 / total
top3_accuracy = 100.0 * correct_top3 / total

print(f"\nFinal Test Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Final Test Top-3 Accuracy: {top3_accuracy:.2f}%")

# ----- 13. Predict on New Samples -----
new_samples = np.array([
    [-2.63, 3.44],
    [-1.53, 3.83]
])
new_samples_scaled = scaler.transform(new_samples)
new_tensor = torch.tensor(new_samples_scaled, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs = model(new_tensor)
    _, preds = torch.max(outputs, 1)

predicted_labels = label_encoder.inverse_transform(preds.cpu().numpy())
for sample, label in zip(new_samples, predicted_labels):
    print(f"d18O={sample[0]}, d13C={sample[1]} => {label}")

Epoch [1/100], Train Loss: 3.4379, Val Loss: 3.3013, Val Acc: 17.57%
Epoch [2/100], Train Loss: 3.1977, Val Loss: 3.0821, Val Acc: 31.76%
Epoch [3/100], Train Loss: 3.0051, Val Loss: 2.9029, Val Acc: 29.73%
Epoch [4/100], Train Loss: 2.8623, Val Loss: 2.7663, Val Acc: 31.08%
Epoch [5/100], Train Loss: 2.7418, Val Loss: 2.6256, Val Acc: 32.43%
Epoch [6/100], Train Loss: 2.6586, Val Loss: 2.5451, Val Acc: 32.43%
Epoch [7/100], Train Loss: 2.5931, Val Loss: 2.4859, Val Acc: 33.11%
Epoch [8/100], Train Loss: 2.5316, Val Loss: 2.4454, Val Acc: 33.78%
Epoch [9/100], Train Loss: 2.4945, Val Loss: 2.3870, Val Acc: 34.46%
Epoch [10/100], Train Loss: 2.4803, Val Loss: 2.3485, Val Acc: 33.78%
Epoch [11/100], Train Loss: 2.4311, Val Loss: 2.3140, Val Acc: 33.78%
Epoch [12/100], Train Loss: 2.3971, Val Loss: 2.2827, Val Acc: 33.78%
Epoch [13/100], Train Loss: 2.3653, Val Loss: 2.2441, Val Acc: 34.46%
Epoch [14/100], Train Loss: 2.3471, Val Loss: 2.2357, Val Acc: 35.81%
Epoch [15/100], Train Loss: 2